# MVP — Análisis de Sentimiento (Ternario)

## TF-IDF + Regresión Logística + Integración Backend  
### **Modelo Multilingüe (Español + Portugués)**

**Objetivo del proyecto:** construir un modelo de análisis de sentimiento que reciba texto libre
(reseñas, comentarios) y prediga **Negativo / Neutro / Positivo**, entregando un artefacto
`joblib` listo para integración con backend.

**Label:** derivado de `stars`: **1–2 Negativo, 3 Neutro, 4–5 Positivo**.

**Alcance:** se entrena y evalúa sobre un dataset **multilingüe**
filtrando a `language ∈ {"es","pt"}`.

---

### Esta versión (V11) se construye como evolución de los modelos con foco en operación multilingüe:

- Se amplía el alcance a **Español + Portugués** en un **único pipeline** (un solo artefacto `joblib` y una sola API de inferencia).
- Se documenta explícitamente la **consolidación multifuente** (MARC + otras reviews) y la normalización de esquema.
- Se mantiene la estrategia de modelado **interpretable y CPU-friendly** (TF-IDF + Regresión Logística).
- Se preserva el enfoque operativo: **probabilidades calibradas**, `probas` completas, y regla `review_required` basada en umbral de confianza.

# 0) Configuración y reproducibilidad

Este notebook está diseñado para ejecutarse en Google Colab o entorno local (CPU).

Se fija una semilla global (`SEED = 42`) para asegurar reproducibilidad en:
- muestreos y mezclas estratificadas,
- validación cruzada,
- búsqueda aleatoria de hiperparámetros,
- métricas reportadas.

El enfoque prioriza reproducibilidad y trazabilidad por sobre experimentación no controlada.

---

# Setup (importaciones, seed, paths)

**Nota técnica (historial de decisiones)**: En las iteraciones anteriores se evaluó usar librerías como **Scapy** (descartada por no aportar al NLP del MVP), así como enfoques con **spaCy**/lemmatización y **Naive Bayes**; finalmente se mantuvo un pipeline **CPU-friendly e interpretable** con **TF-IDF + Regresión Logística**, complementado con calibración de probabilidades para operación.

In [1]:
import os
import re
import json
import time
import uuid
import random
import pickle
import hashlib
import unicodedata
from dataclasses import dataclass
from typing import Any, Dict, Optional, List

import numpy as np
import pandas as pd
import joblib

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

pd.set_option("display.max_colwidth", 180)

OUT_DIR = "/mnt/data/out"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Carga de datos

Se asume un **schema mínimo por registro**:

- `language` (ej. `"es"`, `"pt"`)
- `stars` (1–5)
- `review_title`
- `review_body`

Los datos se cargan desde archivos CSV separados por split:
TRAIN, VALIDATION y TEST.

Se valida explícitamente la presencia de las columnas requeridas
antes de continuar con el pipeline.

In [ ]:
TRAIN_PATH = "/content/train_es_pt.csv"
VAL_PATH   = "/content/validation_es_pt.csv"
TEST_PATH  = "/content/test_es_pt.csv"

train = pd.read_csv(TRAIN_PATH)
val   = pd.read_csv(VAL_PATH)
test  = pd.read_csv(TEST_PATH)

required = {"stars","review_title","review_body","language"}
for name, df in [("train",train),("val",val),("test",test)]:
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"{name} no tiene columnas requeridas: {missing}")

print("Shapes:", train.shape, val.shape, test.shape)
train.head(10)


Shapes: (344657, 6) (43082, 6) (43083, 6)


,text_raw,stars,sentiment,language,review_title,review_body
0,Decepcion. Calidad un poco baja,3,Neutro,es,Decepcion,Calidad un poco baja
1,Caja fatal. Caja rota y venía en la misma caja ya os imaginais esquinas y fatal cajas fatal y encima le pegaron la pegatina encima de la propia caja. Me hicieron el regalo,1,Negativo,es,Caja fatal,Caja rota y venía en la misma caja ya os imaginais esquinas y fatal cajas fatal y encima le pegaron la pegatina encima de la propia caja. Me hicieron el regalo
2,"Silencioso pero scroll pesado. El ratón es muy silencioso a la hora de hacer click; lo elegí precisamente por eso, ya que lo uso a diario en la biblioteca y necesitaba no moles...",3,Neutro,es,Silencioso pero scroll pesado,"El ratón es muy silencioso a la hora de hacer click; lo elegí precisamente por eso, ya que lo uso a diario en la biblioteca y necesitaba no molestar con él a los demás, y eso d..."
3,. Ótimo deixa o cabelo maravilhoso.,4,Positivo,pt,NaN,Ótimo deixa o cabelo maravilhoso.
4,. Muito bom.,5,Positivo,pt,NaN,Muito bom.
5,"Regular. Completo kit, pero mala calidad, se doblan sin poder hacer su trabajo",2,Negativo,es,Regular,"Completo kit, pero mala calidad, se doblan sin poder hacer su trabajo"
6,. Ótima.,5,Positivo,pt,NaN,Ótima.
7,Ótimo produtos. Meus sobrinhos adoraram o brinquedo e se divertem muito. Indico e recomendo.,5,Positivo,pt,Ótimo produtos,Meus sobrinhos adoraram o brinquedo e se divertem muito. Indico e recomendo.
8,Tamaño OK. Costuras malas. El tamaño es perfecto para llevar las herramientas básicas. La calidad es buena excepto los bolsillos laterales que en 10 minutos de uso ya estaban d...,2,Negativo,es,Tamaño OK. Costuras malas,El tamaño es perfecto para llevar las herramientas básicas. La calidad es buena excepto los bolsillos laterales que en 10 minutos de uso ya estaban descosidos.
9,. Gostei.,5,Positivo,pt,NaN,Gostei.


# 2) Etiquetado y análisis de distribución

En esta etapa:

- Se transforma `stars` en una etiqueta categórica `sentiment`
  (Negativo / Neutro / Positivo).
- Se normaliza el campo `language`.
- Se inspecha la distribución de datos por:
  - idioma,
  - estrellas,
  - sentimiento.

Este análisis permite validar balance de clases
y detectar posibles sesgos antes del modelado.

In [ ]:
def size_report(df: pd.DataFrame, name: str) -> None:
    print(f"\n{name}")
    print("Total registros:", len(df))
    print("Idiomas:")
    print(df["language"].value_counts(dropna=False))
    print("\nStars:")
    print(df["stars"].value_counts(dropna=False).sort_index())

def stars_to_sentiment(stars: int) -> str:
    s = int(stars)
    if s in (1,2): return "Negativo"
    if s == 3:    return "Neutro"
    return "Positivo"

def add_labels(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["stars"] = out["stars"].astype(int)
    out["sentiment"] = out["stars"].map(stars_to_sentiment)
    out["language"] = out["language"].astype(str).str.lower()
    return out

train = add_labels(train)
val   = add_labels(val)
test  = add_labels(test)

size_report(train, "TRAIN")
size_report(val, "VALIDATION (externa)")
size_report(test, "TEST (holdout)")

print("\nDistribución por idioma x sentimiento (TRAIN):")
print(pd.crosstab(train["language"], train["sentiment"], normalize="index").round(3))



TRAIN
Total registros: 344657
Idiomas:
language
pt    184657
es    160000
Name: count, dtype: int64

Stars:
stars
1     55607
2     39975
3     47568
4     62443
5    139064
Name: count, dtype: int64

VALIDATION (externa)
Total registros: 43082
Idiomas:
language
pt    23082
es    20000
Name: count, dtype: int64

Stars:
stars
1     7067
2     4881
3     5946
4     7857
5    17331
Name: count, dtype: int64

TEST (holdout)
Total registros: 43083
Idiomas:
language
pt    23083
es    20000
Name: count, dtype: int64

Stars:
stars
1     7045
2     4903
3     5946
4     7856
5    17333
Name: count, dtype: int64

Distribución por idioma x sentimiento (TRAIN):
sentiment  Negativo  Neutro  Positivo
language                             
es            0.400   0.200     0.400
pt            0.171   0.084     0.745


# 3) Construcción y limpieza de texto

El texto que consume el modelo se construye concatenando:

review_title + ". " + review_body

La limpieza del texto sigue un enfoque **conservador y multilingüe**:

- minúsculas,
- eliminación de URLs, menciones y hashtags,
- preservación de caracteres Unicode (acentos ES/PT),
- normalización de espacios.

No se aplica stemming ni lematización para:
- preservar negaciones,
- evitar dependencias externas,
- facilitar despliegue productivo.

In [ ]:
def build_text_raw(df: pd.DataFrame) -> pd.Series:
    return (
        df["review_title"].fillna("").astype(str).str.strip() + ". " +
        df["review_body"].fillna("").astype(str).str.strip()
    ).str.strip()

_noise_re = re.compile(r"(http\S+|www\.\S+|@\w+|#\w+)", re.IGNORECASE)
_space_re = re.compile(r"\s+")

def _keep_char(ch: str) -> bool:
    # Letras unicode (incluye acentos), números y espacio
    if ch.isspace():
        return True
    cat = unicodedata.category(ch)
    # L* letras, M* marcas (acentos combinados), Nd dígitos
    return cat.startswith(("L","M")) or cat == "Nd"

def clean_text_unicode(text: str) -> str:
    if text is None:
        return ""
    t = str(text).lower()
    t = _noise_re.sub(" ", t)
    t = "".join(ch if _keep_char(ch) else " " for ch in t)
    t = _space_re.sub(" ", t).strip()
    return t

def prepare_multilang(df: pd.DataFrame, langs=("es","pt"), min_len: int = 5) -> pd.DataFrame:
    d = df[df["language"].isin(langs)].copy()
    d["text_raw"] = build_text_raw(d)
    d["text_clean"] = d["text_raw"].map(clean_text_unicode)
    d = d[d["text_clean"].str.len() >= min_len].copy()
    # sentiment ya existe (lo agregamos en add_labels)
    return d[["text_raw","text_clean","sentiment","language","stars"]].copy()

train_p = prepare_multilang(train)
val_p   = prepare_multilang(val)
test_p  = prepare_multilang(test)

print("Preparados:", train_p.shape, val_p.shape, test_p.shape)
train_p.head(3)


Preparados: (341798, 5) (42721, 5) (42675, 5)


,text_raw,text_clean,sentiment,language,stars
0,Decepcion. Calidad un poco baja,decepcion calidad un poco baja,Neutro,es,3
1,Caja fatal. Caja rota y venía en la misma caja ya os imaginais esquinas y fatal cajas fatal y encima le pegaron la pegatina encima de la propia caja. Me hicieron el regalo,caja fatal caja rota y venía en la misma caja ya os imaginais esquinas y fatal cajas fatal y encima le pegaron la pegatina encima de la propia caja me hicieron el regalo,Negativo,es,1
2,"Silencioso pero scroll pesado. El ratón es muy silencioso a la hora de hacer click; lo elegí precisamente por eso, ya que lo uso a diario en la biblioteca y necesitaba no moles...",silencioso pero scroll pesado el ratón es muy silencioso a la hora de hacer click lo elegí precisamente por eso ya que lo uso a diario en la biblioteca y necesitaba no molestar...,Neutro,es,3


# 4) Preparación multilingüe y split interno

Se filtran explícitamente los idiomas soportados (`es`, `pt`)
y se eliminan textos demasiado cortos.

Posteriormente, el conjunto TRAIN se divide en:
- `train_fit`: entrenamiento del modelo
- `calib_set`: calibración de probabilidades y selección de umbral

El split es **estratificado por idioma y sentimiento**
para preservar proporciones.

In [ ]:
train_p["strata"] = train_p["language"] + "||" + train_p["sentiment"]

train_fit, calib_set = train_test_split(
    train_p,
    test_size=0.15,  # 15% para calibración + umbral (ajustable)
    random_state=SEED,
    stratify=train_p["strata"]
)

print("train_fit:", train_fit.shape, "| calib_set:", calib_set.shape)
print(pd.crosstab(calib_set["language"], calib_set["sentiment"], normalize="index").round(3))


train_fit: (290528, 6) | calib_set: (51270, 6)
sentiment  Negativo  Neutro  Positivo
language                             
es            0.400   0.200     0.400
pt            0.173   0.085     0.742


# 5) Pipeline de modelado

El pipeline base está compuesto por:

1. `TextCleaner`: limpieza Unicode consistente
2. `TF-IDF Vectorizer`
3. `Logistic Regression` multiclase

Este enfoque se eligió por:
- interpretabilidad,
- eficiencia en CPU,
- buen rendimiento con texto sparse,
- facilidad de calibración posterior.

In [ ]:
class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return [clean_text_unicode(x) for x in X]

base_pipe = Pipeline(steps=[
    ("cleaner", TextCleaner()),
    ("tfidf", TfidfVectorizer(sublinear_tf=True)),
    ("model", LogisticRegression(
        solver="lbfgs",
        max_iter=4000,
        random_state=SEED,
        class_weight="balanced",
        n_jobs=-1
    ))
])


# 6) Optimización de hiperparámetros

Se utiliza `RandomizedSearchCV` con:

- validación cruzada estratificada (cv=3),
- métrica principal: **F1-macro**,
- espacio acotado de hiperparámetros TF-IDF y LR.

Esta estrategia ofrece una mejor relación
costo/beneficio que GridSearch en datasets grandes.

In [ ]:
X_train = train_fit["text_raw"].tolist()
y_train = train_fit["sentiment"].tolist()

C_candidates = [0.01, 0.03, 0.1, 0.3, 1.0, 2.0, 3.0, 5.0, 10.0]

param_distributions = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [2, 3, 5, 8, 12],
    "tfidf__max_df": [0.90, 0.95, 0.98],
    "tfidf__max_features": [80000, 120000, 200000],
    "model__C": C_candidates,
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

rs = RandomizedSearchCV(
    estimator=base_pipe,
    param_distributions=param_distributions,
    n_iter=25,
    scoring="f1_macro",
    cv=cv,
    random_state=SEED,
    n_jobs=1,
    verbose=1,
    refit=True
)

rs.fit(X_train, y_train)

print("\nBest CV F1-macro:", round(float(rs.best_score_), 4))
print("Best params:", rs.best_params_)

best_pipe = rs.best_estimator_


Fitting 3 folds for each of 25 candidates, totalling 75 fits

Best CV F1-macro: 0.7293
Best params: {'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 3, 'tfidf__max_features': 200000, 'tfidf__max_df': 0.9, 'model__C': 1.0}


# 7) Calibración de probabilidades

El modelo optimizado se calibra usando
`CalibratedClassifierCV` (sigmoid).

La calibración se realiza sobre un conjunto separado
(`calib_set`) para evitar fuga de información.

El objetivo es obtener probabilidades confiables
para decisiones operativas.

In [ ]:
from sklearn.frozen import FrozenEstimator
from sklearn.calibration import CalibratedClassifierCV

X_cal = calib_set["text_raw"].tolist()
y_cal = calib_set["sentiment"].tolist()

calibrated_pipe = CalibratedClassifierCV(
    estimator=FrozenEstimator(best_pipe),
    method="sigmoid"

)
calibrated_pipe.fit(X_cal, y_cal)

labels = list(calibrated_pipe.classes_)
print("Labels:", labels)


Labels: [np.str_('Negativo'), np.str_('Neutro'), np.str_('Positivo')]


# 8) Selección de umbral operativo

La revisión humana no se activa por la clase predicha,
sino por **incertidumbre**.

Regla general:

review_required = max(probabilidades) < threshold

El umbral se selecciona evaluando:
- coverage mínimo requerido,
- F1-macro automático,
- recall de la clase Negativo.

In [ ]:
from sklearn.metrics import classification_report

def threshold_table(y_true, y_pred, proba, classes, thresholds, focus_label="Negativo"):
    y_true = np.array(y_true)
    maxp = proba.max(axis=1)
    rows = []
    for th in thresholds:
        auto = maxp >= th
        coverage = float(auto.mean())
        if auto.sum() == 0:
            rows.append({"threshold": th, "coverage": 0.0, "f1_macro_auto": np.nan, f"recall_{focus_label}_auto": np.nan, "n_auto": 0})
            continue
        y_true_auto = y_true[auto]
        y_pred_auto = np.array(y_pred)[auto]
        f1m = float(f1_score(y_true_auto, y_pred_auto, average="macro"))
        rep = classification_report(y_true_auto, y_pred_auto, output_dict=True, zero_division=0)
        recall_focus = float(rep.get(focus_label, {}).get("recall", np.nan))
        rows.append({
            "threshold": th,
            "coverage": round(coverage, 4),
            "f1_macro_auto": round(f1m, 4),
            f"recall_{focus_label}_auto": round(recall_focus, 4) if not np.isnan(recall_focus) else np.nan,
            "n_auto": int(auto.sum())
        })
    return pd.DataFrame(rows).sort_values("threshold")

cal_proba = calibrated_pipe.predict_proba(X_cal)
cal_pred  = calibrated_pipe.predict(X_cal)

thresholds = [round(x, 2) for x in np.arange(0.50, 0.91, 0.05)]
tab = threshold_table(y_cal, cal_pred, cal_proba, labels, thresholds, focus_label="Negativo")
print(tab)

MIN_COVERAGE = 0.70
cand = tab[tab["coverage"] >= MIN_COVERAGE].copy()
if len(cand) == 0:
    chosen = tab.iloc[tab["f1_macro_auto"].fillna(-1).argmax()]
else:
    cand = cand.sort_values(by=["recall_Negativo_auto","f1_macro_auto"], ascending=False)
    chosen = cand.iloc[0]

THRESHOLD = float(chosen["threshold"])
print("\nTHRESHOLD seleccionado:", THRESHOLD)
print("Resumen:", chosen.to_dict())


   threshold  coverage  f1_macro_auto  recall_Negativo_auto  n_auto
0       0.50    0.9427         0.6724                0.9327   48331
1       0.55    0.9017         0.6694                0.9464   46230
2       0.60    0.8600         0.6668                0.9612   44091
3       0.65    0.8138         0.6643                0.9707   41724
4       0.70    0.7620         0.6629                0.9784   39068
5       0.75    0.6962         0.6622                0.9855   35692
6       0.80    0.6105         0.6608                0.9889   31299
7       0.85    0.4843         0.6605                0.9918   24831
8       0.90    0.3245         0.6663                0.9935   16638

THRESHOLD seleccionado: 0.7
Resumen: {'threshold': 0.7, 'coverage': 0.762, 'f1_macro_auto': 0.6629, 'recall_Negativo_auto': 0.9784, 'n_auto': 39068.0}


# 9) Evaluación final

El modelo se evalúa en:

- VALIDATION externa
- TEST holdout

Se reportan métricas globales y por idioma,
incluyendo matriz de confusión.

In [ ]:
def eval_report(df_eval: pd.DataFrame, name: str, model, threshold: float):
    X = df_eval["text_raw"].tolist()
    y = df_eval["sentiment"].tolist()

    proba = model.predict_proba(X)
    pred  = model.predict(X)
    maxp  = proba.max(axis=1)

    print(f"\n=== {name} (GLOBAL) ===")
    print("Accuracy:", round(float(accuracy_score(y, pred)), 4))
    print("F1-macro:", round(float(f1_score(y, pred, average="macro")), 4))
    print(classification_report(y, pred, zero_division=0))

    review = maxp < threshold
    print("Review_required:", int(review.sum()), "/", len(review), f"({review.mean()*100:.1f}%)")
    print("Auto-coverage:", round(float((~review).mean()), 4))

    # Por idioma
    for lang in sorted(df_eval["language"].unique()):
        d = df_eval[df_eval["language"] == lang].copy()
        Xl = d["text_raw"].tolist()
        yl = d["sentiment"].tolist()
        pl = model.predict(Xl)
        print(f"\n--- {name} | idioma={lang} ---")
        print("N:", len(d))
        print("F1-macro:", round(float(f1_score(yl, pl, average="macro")), 4))
        print(classification_report(yl, pl, zero_division=0))

eval_report(val_p, "VALIDATION externa", calibrated_pipe, THRESHOLD)



=== VALIDATION externa (GLOBAL) ===
Accuracy: 0.8312
F1-macro: 0.6666
              precision    recall  f1-score   support

    Negativo       0.78      0.91      0.84     11942
      Neutro       0.54      0.16      0.25      5937
    Positivo       0.88      0.95      0.91     24842

    accuracy                           0.83     42721
   macro avg       0.73      0.67      0.67     42721
weighted avg       0.80      0.83      0.80     42721

Review_required: 10117 / 42721 (23.7%)
Auto-coverage: 0.7632

--- VALIDATION externa | idioma=es ---
N: 20000
F1-macro: 0.649
              precision    recall  f1-score   support

    Negativo       0.76      0.91      0.83      8000
      Neutro       0.52      0.18      0.27      4000
    Positivo       0.80      0.90      0.85      8000

    accuracy                           0.76     20000
   macro avg       0.69      0.67      0.65     20000
weighted avg       0.73      0.76      0.72     20000


--- VALIDATION externa | idioma=pt ---
N

In [ ]:
eval_report(test_p, "TEST (holdout)", calibrated_pipe, THRESHOLD)

# Matriz de confusión global (TEST)
X_test = test_p["text_raw"].tolist()
y_test = test_p["sentiment"].tolist()
pred_test = calibrated_pipe.predict(X_test)

cm = confusion_matrix(y_test, pred_test, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("\nMatriz de confusión (TEST, global):")
cm_df



=== TEST (holdout) (GLOBAL) ===
Accuracy: 0.8333
F1-macro: 0.6729
              precision    recall  f1-score   support

    Negativo       0.79      0.91      0.84     11943
      Neutro       0.56      0.17      0.26      5925
    Positivo       0.88      0.95      0.91     24807

    accuracy                           0.83     42675
   macro avg       0.74      0.68      0.67     42675
weighted avg       0.81      0.83      0.80     42675

Review_required: 9996 / 42675 (23.4%)
Auto-coverage: 0.7658

--- TEST (holdout) | idioma=es ---
N: 20000
F1-macro: 0.6552
              precision    recall  f1-score   support

    Negativo       0.76      0.91      0.83      8000
      Neutro       0.56      0.19      0.29      4000
    Positivo       0.80      0.91      0.85      8000

    accuracy                           0.77     20000
   macro avg       0.71      0.67      0.66     20000
weighted avg       0.74      0.77      0.73     20000


--- TEST (holdout) | idioma=pt ---
N: 22675
F1-m

,Negativo,Neutro,Positivo
Negativo,10890,353,700
Neutro,2263,1010,2652
Positivo,719,429,23659


# Interpretabilidad del modelo

Para explicar el comportamiento del modelo:

- se reentrena un pipeline no calibrado,
- se extraen términos más influyentes por clase
  a partir de los coeficientes del modelo lineal.

La calibración se usa solo para probabilidades,
no para explicación.

In [ ]:
best_params = rs.best_params_

explain_pipe = Pipeline(steps=[
    ("cleaner", TextCleaner()),
    ("tfidf", TfidfVectorizer(
        sublinear_tf=True,
        ngram_range=best_params.get("tfidf__ngram_range", (1,2)),
        min_df=best_params.get("tfidf__min_df", 3),
        max_df=best_params.get("tfidf__max_df", 0.95),
        max_features=best_params.get("tfidf__max_features", 200000),
    )),
    ("model", LogisticRegression(
        solver="lbfgs",
        max_iter=5000,
        random_state=SEED,
        class_weight="balanced",
        n_jobs=-1,
        C=best_params.get("model__C", 1.0),
    ))
])

explain_pipe.fit(train_fit["text_raw"].tolist(), train_fit["sentiment"].tolist())

def top_terms_per_class_from_lr(pipe: Pipeline, top_k: int = 12) -> Dict[str, Dict[str, List[str]]]:
    vec: TfidfVectorizer = pipe.named_steps["tfidf"]
    lr: LogisticRegression = pipe.named_steps["model"]

    feature_names = np.array(vec.get_feature_names_out())
    classes = lr.classes_
    coefs = lr.coef_

    out = {}
    for i, cls in enumerate(classes):
        top_pos_idx = np.argsort(coefs[i])[-top_k:][::-1]
        top_neg_idx = np.argsort(coefs[i])[:top_k]
        out[str(cls)] = {
            "top_positive": feature_names[top_pos_idx].tolist(),
            "top_negative": feature_names[top_neg_idx].tolist(),
        }
    return out

top_terms = top_terms_per_class_from_lr(explain_pipe, top_k=12)
print(json.dumps(top_terms, ensure_ascii=False, indent=2)[:1500])


{
  "Negativo": {
    "top_positive": [
      "no",
      "não recomendo",
      "não",
      "dos estrellas",
      "péssimo",
      "no funciona",
      "ruim",
      "no cumple",
      "mala",
      "não gostei",
      "decepción",
      "fatal"
    ],
    "top_negative": [
      "bom",
      "excelente",
      "perfecto",
      "gostei",
      "ótimo",
      "buena",
      "buen",
      "un poco",
      "perfectamente",
      "genial",
      "ótima",
      "bien"
    ]
  },
  "Neutro": {
    "top_positive": [
      "tres estrellas",
      "pero",
      "bom",
      "produto bom",
      "bom produto",
      "aceptable",
      "bom excelente",
      "correcto",
      "mejorable",
      "normalito",
      "porém",
      "bien"
    ],
    "top_negative": [
      "dos estrellas",
      "produto",
      "maravilhoso",
      "excelente",
      "recomiendo",
      "estafa",
      "amei",
      "não recomendo",
      "estrella",
      "péssimo",
      "nada recomendable",
      "simplesment

# Serialización y contrato backend

Se genera un artefacto `joblib` que contiene:

- modelo calibrado,
- threshold,
- labels,
- metadata,
- términos explicativos.

Se definen dos contratos:
- interno (trazabilidad completa),
- externo (respuesta mínima estable).

In [ ]:
@dataclass
class ModelMeta:
    model_version: str
    threshold: float
    labels: List[str]
    artifact_hash: str
    best_params: Dict[str, Any]
    train_size: int
    calib_size: int
    val_size: int
    test_size: int

def compute_artifact_hash(obj) -> str:
    blob = pickle.dumps(obj)
    return hashlib.sha256(blob).hexdigest()[:12]

MODEL_VERSION = "sentiment_es_pt_tfidf_lr_calibrated_rs_v2"

bundle_for_hash = {
    "model": calibrated_pipe,
    "threshold": THRESHOLD,
    "labels": labels,
    "best_params": rs.best_params_,
    "top_terms": top_terms,
    "model_version": MODEL_VERSION
}
artifact_hash = compute_artifact_hash(bundle_for_hash)

meta = ModelMeta(
    model_version=MODEL_VERSION,
    threshold=float(THRESHOLD),
    labels=labels,
    artifact_hash=artifact_hash,
    best_params=rs.best_params_,
    train_size=int(len(train_fit)),
    calib_size=int(len(calib_set)),
    val_size=int(len(val_p)),
    test_size=int(len(test_p)),
)

bundle = {
    "model": calibrated_pipe,
    "threshold": float(THRESHOLD),
    "labels": labels,
    "meta": meta.__dict__,
    "top_terms_global": top_terms,
    "explain_pipe": explain_pipe,
}

MODEL_PATH = os.path.join(OUT_DIR, "sentiment_bundle_es_pt_v2.joblib")
joblib.dump(bundle, MODEL_PATH)

print("Guardado:", MODEL_PATH)
print("Meta:", meta)


Guardado: /mnt/data/out/sentiment_bundle_es_pt_v2.joblib
Meta: ModelMeta(model_version='sentiment_es_pt_tfidf_lr_calibrated_rs_v2', threshold=0.7, labels=[np.str_('Negativo'), np.str_('Neutro'), np.str_('Positivo')], artifact_hash='bfd753e81b08', best_params={'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 3, 'tfidf__max_features': 200000, 'tfidf__max_df': 0.9, 'model__C': 1.0}, train_size=290528, calib_size=51270, val_size=42721, test_size=42675)


In [ ]:
# =========================
# 1) Guardar el artefacto en Colab (y opcionalmente en /mnt/data/out)
# =========================
import os
import joblib

MODEL_FILENAME = "sentiment_bundle_es_pt_v2.joblib"

# Ruta recomendada Colab
COLAB_OUT_DIR = "/content/models"
os.makedirs(COLAB_OUT_DIR, exist_ok=True)
MODEL_PATH_COLAB = os.path.join(COLAB_OUT_DIR, MODEL_FILENAME)

joblib.dump(bundle, MODEL_PATH_COLAB)
print("Guardado en Colab:", MODEL_PATH_COLAB)

# (Opcional) mantener también tu ruta /mnt/data/out si la sigues usando
OUT_DIR = "/mnt/data/out"
os.makedirs(OUT_DIR, exist_ok=True)
MODEL_PATH_MNT = os.path.join(OUT_DIR, MODEL_FILENAME)
joblib.dump(bundle, MODEL_PATH_MNT)
print("Guardado en /mnt/data:", MODEL_PATH_MNT)


# =========================
# 2) Contrato interno vs externo
# =========================
import time
import uuid
import numpy as np
from typing import Any, Dict, Optional

def validate_text(text: Any, min_len: int = 5, max_len: int = 2000) -> Optional[Dict[str, str]]:
    if text is None:
        return {"error": "invalid_input", "detail": "text is required"}
    if not isinstance(text, str):
        return {"error": "invalid_input", "detail": "text must be a string"}
    t = text.strip()
    if len(t) < min_len:
        return {"error": "invalid_input", "detail": f"text must have at least {min_len} characters"}
    if len(t) > max_len:
        return {"error": "invalid_input", "detail": f"text must have at most {max_len} characters"}
    return None

def predict_internal(text: str, loaded_bundle: Dict[str, Any]) -> Dict[str, Any]:
    """
    CONTRATO INTERNO: respuesta completa para trazabilidad/observabilidad.
    Ideal para logs, DB, auditoría y debugging.
    """
    err = validate_text(text)
    if err:
        return err

    start = time.perf_counter()
    request_id = str(uuid.uuid4())

    model = loaded_bundle["model"]
    threshold = float(loaded_bundle["threshold"])
    meta = loaded_bundle["meta"]

    proba = model.predict_proba([text])[0]
    classes = list(model.classes_)
    i = int(np.argmax(proba))
    pred = str(classes[i])
    max_prob = float(proba[i])

    probas_dict = {str(c): float(round(p, 6)) for c, p in zip(classes, proba)}
    latency_ms = (time.perf_counter() - start) * 1000.0

    return {
        "request_id": request_id,
        "prevision": pred,
        "probabilidad": round(max_prob, 6),
        "probas": probas_dict,
        "review_required": bool(max_prob < threshold),
        "threshold": threshold,
        "model_version": str(meta.get("model_version")),
        "artifact_hash": str(meta.get("artifact_hash")),
        "latency_ms": round(float(latency_ms), 3),
    }

def to_external_response(internal: Dict[str, Any]) -> Dict[str, Any]:
    """
    CONTRATO EXTERNO: respuesta mínima y estable para consumidores.
    (Backend/Spring, clientes, etc.)
    """
    # Si vino error de validación, lo devolvemos tal cual
    if "error" in internal:
        return internal

    return {
        "prevision": internal["prevision"],
        "probabilidad": internal["probabilidad"],
        "review_required": internal["review_required"],
    }

def predict_external(text: str, loaded_bundle: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper: ejecuta predicción y devuelve SOLO contrato externo.
    """
    internal = predict_internal(text, loaded_bundle)
    return to_external_response(internal)


# =========================
# 3) Smoke test (interno vs externo)
# =========================
loaded_bundle = joblib.load(MODEL_PATH_COLAB)

samples = [
    "El producto es excelente y llegó rápido, muy recomendado.",
    "Está bien, cumple, nada especial.",
    "No funciona, llegó roto y el soporte no responde.",
    "Produto excelente, entrega rápida e recomendo.",
    "Não funciona, veio quebrado e o suporte não responde."
]

print("\n--- CONTRATO INTERNO ---")
for s in samples:
    print(predict_internal(s, loaded_bundle))

print("\n--- CONTRATO EXTERNO ---")
for s in samples:
    print(predict_external(s, loaded_bundle))


Guardado en Colab: /content/models/sentiment_bundle_es_pt_v2.joblib
Guardado en /mnt/data: /mnt/data/out/sentiment_bundle_es_pt_v2.joblib

--- CONTRATO INTERNO ---
{'request_id': '0566ea25-f21c-4b01-87a1-f92d63fa7c50', 'prevision': 'Positivo', 'probabilidad': 0.95674, 'probas': {'Negativo': 0.012809, 'Neutro': 0.030451, 'Positivo': 0.95674}, 'review_required': False, 'threshold': 0.7, 'model_version': 'sentiment_es_pt_tfidf_lr_calibrated_rs_v2', 'artifact_hash': 'bfd753e81b08', 'latency_ms': 5.444}
{'request_id': '8e3ed362-0212-45c7-b486-46f2206043d7', 'prevision': 'Neutro', 'probabilidad': 0.570183, 'probas': {'Negativo': 0.014232, 'Neutro': 0.570183, 'Positivo': 0.415584}, 'review_required': True, 'threshold': 0.7, 'model_version': 'sentiment_es_pt_tfidf_lr_calibrated_rs_v2', 'artifact_hash': 'bfd753e81b08', 'latency_ms': 4.224}
{'request_id': 'fd165ce7-82b4-4cc4-9b01-077275edf34e', 'prevision': 'Negativo', 'probabilidad': 0.871975, 'probas': {'Negativo': 0.871975, 'Neutro': 0.125281